In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import sys
from sklearn import metrics # for the evaluation
from settings import CORR_GROUP, AD_THRESHOLD
from keras.callbacks import EarlyStopping
import tensorflow as tf
import logging

2022-07-13 15:58:42.437092: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-13 15:58:42.437188: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
def unique_cols(df):
    a = df.to_numpy() # df.values (pandas<0.24)
    return (a[0] == a).all(0)


def create_supervised_dataset(df, target, feats, n_in=1, n_out=1):
    cols, names = list(), list()
    n_vars = len(feats)
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df[feats].shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df[target].shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(1)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(1)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    agg.dropna(inplace=True)
    return agg.values

In [3]:
from settings import INPUT_FILE

df = pd.read_csv(INPUT_FILE, index_col='ts')
#df = df.drop('Unnamed: 0', 1)
df.index = pd.to_datetime(df.index)
df_2 = df.loc[:,np.invert(unique_cols(df))]
# Average window
# df_2 = df.groupby(np.arange(len(df))//60).mean()
scaler = MinMaxScaler()
d = scaler.fit_transform(df_2)
scaled_df = pd.DataFrame(d, columns=df_2.columns, index=df_2.index)


In [11]:
scaled_df.head()

,AE_L1,AE_L2,AE_L3,AE_SUM,C_phi_L1,C_phi_L2,C_phi_L3,F,H_TDH_I_L1_N,H_TDH_I_L2_N,...,S_L1,S_L2,S_L3,S_SUM,U_L1_L2,U_L1_N,U_L2_L3,U_L2_N,U_L3_L1,U_L3_N
ts,,,,,,,,,,,,,,,,,,,,,
2022-04-14 09:45:00,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.985689,0.591371,0.878465,0.769139,...,0.0,0.0,0.004865,0.001578,0.600407,0.705059,0.658637,0.614509,0.728167,0.672975
2022-04-14 09:46:00,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.836782,0.632546,0.878465,0.769139,...,0.0,0.0,0.004061,0.001318,0.601648,0.685304,0.636376,0.610935,0.707506,0.654408
2022-04-14 09:47:00,0.0,0.0,4.502219e-07,0.0,0.0,0.0,0.820093,0.603675,0.878465,0.769139,...,0.0,0.0,0.003887,0.001261,0.606283,0.693359,0.635031,0.610293,0.716453,0.659068
2022-04-14 09:48:00,0.0,0.0,4.911512e-07,0.0,0.0,0.0,0.739987,0.597113,0.878465,0.769139,...,0.0,0.0,0.003586,0.001163,0.628999,0.706141,0.650733,0.629983,0.731549,0.679870
2022-04-14 09:49:00,0.0,0.0,7.530985e-07,0.0,0.0,0.0,0.756175,0.536745,0.878465,0.769139,...,0.0,0.0,0.003666,0.001189,0.634475,0.702842,0.665284,0.641441,0.727198,0.687478


In [4]:
for k in CORR_GROUP:
    scaled_df['AD'] = " "
    scaled_df[k + ' AD Detected'] = " "
scaled_df.head()

,AE_L1,AE_L2,AE_L3,AE_SUM,C_phi_L1,C_phi_L2,C_phi_L3,F,H_TDH_I_L1_N,H_TDH_I_L2_N,...,P_SUM AD Detected,U_L1_N AD Detected,I_SUM AD Detected,H_TDH_I_L3_N AD Detected,F AD Detected,ReacEc_L1 AD Detected,C_phi_L3 AD Detected,ReacEc_L3 AD Detected,RealE_SUM AD Detected,H_TDH_U_L2_N AD Detected
ts,,,,,,,,,,,,,,,,,,,,,
2022-04-14 09:45:00,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.985689,0.591371,0.878465,0.769139,...,,,,,,,,,,
2022-04-14 09:46:00,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.836782,0.632546,0.878465,0.769139,...,,,,,,,,,,
2022-04-14 09:47:00,0.0,0.0,4.502219e-07,0.0,0.0,0.0,0.820093,0.603675,0.878465,0.769139,...,,,,,,,,,,
2022-04-14 09:48:00,0.0,0.0,4.911512e-07,0.0,0.0,0.0,0.739987,0.597113,0.878465,0.769139,...,,,,,,,,,,
2022-04-14 09:49:00,0.0,0.0,7.530985e-07,0.0,0.0,0.0,0.756175,0.536745,0.878465,0.769139,...,,,,,,,,,,


In [8]:
import random
from settings import SEED1
random.seed(SEED1)

anomaly_df = scaled_df.tail(int(0.1*len(scaled_df)))
control = 0
for index, row in anomaly_df.iterrows():
    if control != 0:
        if control > 5:
            anomaly_df.at[index, 'AD'] = True
            for k in CORR_GROUP:
                anomaly_df.at[index, k] /= 2
            control = 0
            continue

        anomaly_df.at[index, 'AD'] = True
        for k in CORR_GROUP:
            anomaly_df.at[index, k] = 0
        control += 1
        continue

    is_anomaly = random.random() < 0.01
    if is_anomaly:
        anomaly_df.at[index, 'AD'] = True
        for k in CORR_GROUP:
            anomaly_df.at[index, k] /= 2
        control += 1
        
    else:
        anomaly_df.at[index, 'AD'] = False

scaled_df.iloc[-int(0.1*len(scaled_df)):] = anomaly_df
scaled_df.tail()

AE_L1     AE_L2     AE_L3    AE_SUM  C_phi_L1  \
ts                                                                      
2022-05-11 20:54:00  0.999940  0.999937  0.999939  0.999939  0.964400   
2022-05-11 20:55:00  0.999971  0.999969  0.999970  0.999970  0.964736   
2022-05-11 20:56:00  0.999995  0.999993  0.999994  0.999994  0.764903   
2022-05-11 20:57:00  1.000000  1.000000  1.000000  1.000000  0.000000   
2022-05-11 20:58:00  1.000000  1.000000  1.000000  1.000000  0.000000   

                     C_phi_L2  C_phi_L3         F  H_TDH_I_L1_N  H_TDH_I_L2_N  \
ts                                                                              
2022-05-11 20:54:00  0.974470  0.461782  0.520997      0.534157      0.473966   
2022-05-11 20:55:00  0.978975  0.960113  0.506562      0.522807      0.462906   
2022-05-11 20:56:00  0.866177  0.971629  0.525591      0.737308      0.592497   
2022-05-11 20:57:00  0.431670  0.992824  0.498031      0.878465      0.409323   
2022-05-11 20:58:00  0.414770  0.993229  0.517435      0.878465      0.384148   

                     ...  ReacEc_L1 AD  ReacEc_L1 AD Detected  C_phi_L3 AD  \
ts                   ...                                                     
2022-05-11 20:54:00  ...         False                                True   
2022-05-11 20:55:00  ...         False                               False   
2022-05-11 20:56:00  ...         False                               False   
2022-05-11 20:57:00  ...         False                               False   
2022-05-11 20:58:00  ...         False                               False   

                     C_phi_L3 AD Detected  ReacEc_L3 AD  \
ts                                                        
2022-05-11 20:54:00                               False   
2022-05-11 20:55:00                               False   
2022-05-11 20:56:00                               False   
2022-05-11 20:57:00                               False   
2022-05-11 20:58:00                                True   

                     ReacEc_L3 AD Detected  RealE_SUM AD  \
ts                                                         
2022-05-11 20:54:00                                False   
2022-05-11 20:55:00                                False   
2022-05-11 20:56:00                                False   
2022-05-11 20:57:00                                False   
2022-05-11 20:58:00                                False   

                     C_phi_L3 AD Detected  ReacEc_L3 AD Detected  \
ts                                                                 
2022-05-11 20:54:00                                                
2022-05-11 20:55:00                                                
2022-05-11 20:56:00                                                
2022-05-11 20:57:00                                                
2022-05-11 20:58:00                                                

                     RealE_SUM AD Detected  H_TDH_U_L2_N AD Detected  
ts                                                                    
2022-05-11 20:54:00                                                   
2022-05-11 20:55:00                                                   
2022-05-11 20:56:00                                                   
2022-05-11 20:57:00                                                   
2022-05-11 20:58:00                                                   

[5 rows x 67 columns]

In [15]:
import pandas as pd
import tensorflow as tf

import autokeras as ak

from settings import INPUT_FILE, CORR_GROUP, OUTPUT_FILE
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import logging

for var in CORR_GROUP:
    print(var)
    model = tf.keras.models.load_model(f'models/{var}_autokeras.h5')
    features = []
    counter = 0
    history_window = 15
    for index, row in scaled_df.iterrows():

        if counter >= history_window:
            if row[var + ' AD'] != True or row[var + ' AD'] != False:
                tensor = np.array(features).reshape(-1, history_window, len(CORR_GROUP[var]))
                res = model.predict(tensor, verbose=0)
                ad_detected = abs(res - row[var]) > AD_THRESHOLD[var]
                scaled_df.at[index, var + ' AD Detected'] = ad_detected
            features = features[len(CORR_GROUP[var]):]
            
        counter += 1
        predictors = row[CORR_GROUP[var]]
        features += predictors.to_list()
    break
    

P_SUM


2022-07-12 14:06:55.112467: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-07-12 14:06:55.112654: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-07-12 14:06:55.112736: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (semantic): /proc/driver/nvidia/version does not exist
2022-07-12 14:06:55.113526: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


U_L1_N


Exception ignored in: <function ScopedTFGraph.__del__ at 0x7f741d9568b0>
Traceback (most recent call last):
  File "/home/atnoguser/git/Electricity-Forecasting/venv/lib/python3.8/site-packages/tensorflow/python/framework/c_api_util.py", line 55, in __del__
    self.deleter(self.graph)
KeyboardInterrupt: 


In [30]:
from tensorflow import keras
from AttentionBiLSTM import attention

for var in ['U_L1_N', 'I_SUM', 'F', 'C_phi_L3']:
    model = keras.models.load_model(f'models/{var}_autokeras.h5')
    features = []
    counter = 0
    history_window = 15
    for index, row in scaled_df.iterrows():
        if counter >= history_window:
            tensor = np.array(features).reshape((1, 1, len(features)))
            res = model.predict(tensor)
            ad_detected = abs(res - row[var]) > AD_THRESHOLD[var]
            scaled_df.at[index, var + ' AD'] = ad_detected
            features = features[len(CORR_GROUP[var]):]

        counter += 1
        predictors = row[CORR_GROUP[var]]
        features += predictors.to_list()
    

In [31]:
for i in ['P_SUM', 'U_L1_N', 'I_SUM', 'F', 'C_phi_L3']:
    df[i + ' AD'] = scaled_df[i + ' AD']

df.to_csv('data/anomaly_detection_data.csv')

In [4]:
1 + False

1